# Prep data

This is unnecessary to run if you have the "case_study.csv" dataset locally.

In [1]:
import pandas as pd
from sublimpy import tidy, utils

In [2]:
start_date = '20221130'
end_date = '20230509'

# open files
tidy_df = pd.read_parquet(f'../paper1/tidy_df_{start_date}_{end_date}_noplanar_fit.parquet')

# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, 'UTC', 'US/Mountain')

# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df = tidy_df.set_index('time').sort_index().loc[start_date:end_date].reset_index()

## Add combined blowing snow flux variable
tidy_df = tidy.tidy_df_add_variable(
    tidy_df,
    (
        tidy_df.query("variable == 'SF_avg_1m_ue'")['value'].values + 
        tidy_df.query("variable == 'SF_avg_2m_ue'")['value'].values
    ), 
    'SF_avg_ue',
    'snow flux',
    1,
    'ue',
)

In [3]:
case_src = tidy_df[(tidy_df.time >= '4/15/2023') & (tidy_df.time < '4/20/2023')]
case_src = case_src[case_src.measurement.isin([
    'temperature', 'surface temperature', 'relative humidity', 'specific humidity', 'relative humidity', 'w_h2o_', 'snow flux', 'snow depth', 'wind speed','irgason diagnostic flag', 'sonic anemometer diagnostic flag'
])]
case_src = case_src[~ case_src.variable.str.contains("predicted")]

In [4]:
case_src_wide = case_src.pivot_table(
    index = 'time',
    columns='variable',
    values='value'
).reset_index()

In [5]:
case_src.to_csv("case_study_tidy.csv", index=False)

In [6]:
case_src_wide.to_csv("case_study_wide.csv", index=False)

In [11]:
import altair as alt

In [30]:

src = tidy_df[tidy_df.variable.isin([
    'ldiag_3m_c', 'ldiag_10m_c',
    'T_3m_c', 'RH_3m_c', 'spd_3m_c', 'SF_avg_ue'
])]
alt.Chart(
    src.pivot_table(index='time', columns='variable', values='value').reset_index()
).transform_fold([
    'T_3m_c', 'RH_3m_c', 'spd_3m_c', 'SF_avg_ue'
]).mark_circle(size=10).encode(
    alt.X('ldiag_3m_c:Q').scale(type='symlog').axis(values=[0,10,100,1000]),
    alt.Y('value:Q'),
    alt.Facet('key:N', columns=4)
).properties(width=200, height = 200).resolve_scale(y='independent').configure_axis(grid=False)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [58]:
import xarray as xr
import datetime as dt
precip_df = xr.open_dataset("/Users/elischwat/Development/data/sublimationofsnow/precip_danny/precipitation_rate_gts_w23.nc")['corrected_prcp_rate_m2'].to_dataframe()

is_snowing_dates = pd.concat([
    precip_df.query("corrected_prcp_rate_m2 > 0").index.to_series(),
    precip_df.query("corrected_prcp_rate_m2 > 0").index.to_series() + dt.timedelta(minutes=30)
])
    
is_not_snowing_dates = pd.concat([
    precip_df.query("corrected_prcp_rate_m2 <= 0").index.to_series(),
    precip_df.query("corrected_prcp_rate_m2 <= 0").index.to_series() + dt.timedelta(minutes=30)
])

In [64]:
src = tidy_df[tidy_df.variable.isin([
    'ldiag_3m_c', 'ldiag_10m_c',
    'T_3m_c', 'RH_3m_c', 'spd_3m_c', 'SF_avg_ue'
])]
src = src.pivot_table(index='time', columns='variable', values='value').reset_index()
src = src[src.time <= precip_df.index.max()]

alt.Chart(
    src
).transform_calculate(
    is_bs = "datum.SF_avg_ue > 0"
).mark_boxplot(size=10).encode(
    alt.Y('is_bs:N'),
    alt.X('ldiag_3m_c:Q').scale(type='symlog').axis(values=[0,10,100,1000]),
).properties(width=1000, height = 100).resolve_scale(y='independent') &\
alt.Chart(
    src[src.time.isin(is_not_snowing_dates)]
).transform_calculate(
    is_bs = "datum.SF_avg_ue > 0"
).mark_boxplot(size=10).encode(
    alt.Y('is_bs:N'),
    alt.X('ldiag_3m_c:Q').scale(type='symlog').axis(values=[0,10,100,1000]),
).properties(width=1000, height = 100).resolve_scale(y='independent')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [68]:
src[src.time.isin(is_not_snowing_dates)].query("ldiag_3m_c>0").query("SF_avg_ue > 0")

variable,time,RH_3m_c,SF_avg_ue,T_3m_c,ldiag_10m_c,ldiag_3m_c,spd_3m_c
99,2022-12-02 01:30:00,82.697914,0.010000,-2.935653,11.000000,3.000000,2.626857
100,2022-12-02 02:00:00,90.262970,0.060000,-3.553481,8.166667,2.000000,3.266315
102,2022-12-02 03:00:00,91.015106,0.006667,-3.527881,4.666667,1.000000,2.025441
103,2022-12-02 03:30:00,79.646782,0.003333,-2.805454,5.000000,2.000000,2.117924
105,2022-12-02 04:30:00,71.255684,0.001667,-2.698964,21.333333,1.000000,2.983412
...,...,...,...,...,...,...,...
5795,2023-03-30 18:30:00,68.807076,0.001667,-5.125519,7.166667,4.166667,3.091792
5797,2023-03-30 19:30:00,70.765968,0.001667,-5.818987,18.500000,2.000000,3.053634
5828,2023-03-31 11:00:00,77.502373,0.053333,-8.325741,271.000000,54.333333,5.993345
5829,2023-03-31 11:30:00,79.509026,0.013333,-8.190660,122.500000,29.333333,4.781885


In [66]:
src[src.time.isin(is_not_snowing_dates)].query("ldiag_3m_c>0").query("SF_avg_ue == 0")

variable,time,RH_3m_c,SF_avg_ue,T_3m_c,ldiag_10m_c,ldiag_3m_c,spd_3m_c
85,2022-12-01 18:30:00,42.308662,0.0,1.264583,2.000000,1.166667,2.914105
93,2022-12-01 22:30:00,49.916645,0.0,-0.289065,0.000000,1.166667,3.438851
96,2022-12-02 00:00:00,44.218521,0.0,0.820055,3.000000,1.000000,4.414920
97,2022-12-02 00:30:00,57.638187,0.0,-0.616042,13.333333,1.166667,4.393628
163,2022-12-03 09:30:00,85.407867,0.0,-8.150525,2.000000,3.000000,1.080094
...,...,...,...,...,...,...,...
5597,2023-03-26 15:30:00,83.037292,0.0,-10.513554,0.000000,1.000000,2.452425
5598,2023-03-26 16:00:00,79.620819,0.0,-10.358121,2.000000,1.333333,1.890710
5792,2023-03-30 17:00:00,85.635063,0.0,-5.908267,25.500000,3.333333,2.913078
5794,2023-03-30 18:00:00,68.487526,0.0,-4.924978,5.166667,2.166667,3.619690


# Run analysis

Read in "case_study.csv" and analyze the data

In [7]:
import altair as alt
import pandas as pd
alt.data_transformers.enable('json')
alt.renderers.enable('jupyterlab')

RendererRegistry.enable('jupyterlab')

In [8]:
case_src = pd.read_csv("case_study_tidy.csv")
case_src.time = pd.to_datetime(case_src.time)
case_src.head()

,time,variable,value,height,tower,measurement
0,2023-04-15,w_h2o__1m_ue,0.000000,1.0,ue,w_h2o_
1,2023-04-15,irgadiag_20m_c,37.333333,20.0,c,irgason diagnostic flag
2,2023-04-15,irgadiag_3m_uw,1107.833333,3.0,uw,irgason diagnostic flag
3,2023-04-15,T_18m_c,-5.653895,18.0,c,temperature
4,2023-04-15,irgadiag_10m_ue,168.000000,10.0,ue,irgason diagnostic flag


In [9]:
alt.Chart(
    case_src[case_src.variable.isin([
        'ldiag_2m_c',
        'ldiag_3m_c',
        'ldiag_10m_c',
        
        'ldiag_3m_ue', 'ldiag_3m_uw', 'ldiag_3m_ud',
        'ldiag_10m_ue', 'ldiag_10m_uw', 'ldiag_10m_ud',
    ])]
).mark_line().encode(
    alt.X("time:T"),
    alt.Y("value:Q").title(
        "Sonic diag. flag count"
    ).scale(type='symlog').axis(values=[0,10,100,200]),
    alt.Color('height:O').title('height (m)'),
    alt.Row('tower:N')
).properties(height = 150, width = 300).configure_legend(
    fillColor='white', strokeWidth=1, strokeColor='black', padding=10,
    orient='none',  
    legendX=70,
    legendY=10
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [48]:
(alt.Chart(
        case_src[case_src.variable.isin(['SF_avg_ue', 'spd_3m_c'])]
    ).mark_line().encode(
        alt.X("time:T"),
        alt.Y("value:Q").title(
            "Wind speed (m/s) / Blowing snow flux (g/m^2/s)"
        ),
        alt.Color('variable:N')
    ).properties(height = 200) & alt.Chart(
        case_src[case_src.variable.isin(['Tsurf_c', 'Tsurf_d', 'Tsurf_ue', 'Tsurf_uw', 'Tsurf_rad_d'])]
    ).mark_line().encode(
        alt.X("time:T"),
        alt.Y("value:Q").title(
            "Surface temperature (˚C)"
        ),
        alt.Color('variable:N')
    ).properties(height = 200) ).resolve_scale(color='independent')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [11]:
bs_flux_and_snow_depth_chart = (
    alt.Chart(
        case_src.query("variable == 'SF_avg_ue'")
    ).mark_line().encode(
        alt.X("time:T"),
        alt.Y("value:Q").scale(type='symlog').title(
            "Blowing snow flux (g/m^2/s)"
        )
    ).properties(height = 200) 
    + 
    alt.Chart(
        case_src.query("variable == 'SnowDepth_c'")
    ).mark_line().encode(
        alt.X("time:T"),
        alt.Y("value:Q").scale(type='symlog').title(
            "Snow depth (m)"
        )
    ).properties(height = 200)
).resolve_scale(y='independent')

In [5]:
hl_flux_chart = alt.Chart(
    case_src[
        case_src.time.dt.minute == 0
    ][
        case_src.time.dt.hour % 2 == 0
    ].query(
        "measurement == 'w_h2o_'"
    ).query(
        "tower == 'c'"
    ).query("height > 1")
).mark_line().encode(
    alt.X("value:Q").sort('-y').title("<w'q'> (g/m²/s)"),
    alt.Y("height:Q").title("Height (m)"),
    alt.Color("hours(time):O").scale(scheme='rainbow'),
    alt.Facet("date(time):T")
).properties(
    width=200, 
    height = 200
)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_40419/1771112374.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  case_src[


In [6]:
temp_chart = alt.Chart(
    case_src[
        case_src.time.dt.minute == 0
    ][
        case_src.time.dt.hour % 2 == 0
    ][
        case_src.measurement.isin(['temperature', 'surface temperature'])
    ].query("tower == 'c'")
).mark_line().encode(
    alt.X("value:Q").sort('-y').title("Temperature (˚C)"),
    alt.Y("height:Q").title("Height (m)"),
    alt.Color("hours(time):O").scale(scheme='rainbow'),
    alt.Facet("date(time):T")
).properties(
    width=200, 
    height = 200
)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_40419/2652316108.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  case_src[
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_40419/2652316108.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  case_src[


In [7]:
spechumidity_chart = alt.Chart(
    case_src[
        case_src.time.dt.minute == 0
    ][
        case_src.time.dt.hour % 2 == 0
    ].query(
        "measurement == 'specific humidity'"
    ).query(
        "tower == 'c'"
    )
).mark_line().encode(
    alt.X("value:Q").sort('-y').title("Specific humidity (g/g)"),
    alt.Y("height:Q").title("Height (m)"),
    alt.Color("hours(time):O").scale(scheme='rainbow'),
    alt.Facet("date(time):T")
).properties(
    width=200, 
    height = 200
)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_40419/1741176887.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  case_src[


In [8]:
combo_chart = (
    bs_flux_and_snow_depth_chart & hl_flux_chart & temp_chart & spechumidity_chart
).resolve_scale(
    color='independent'
)
combo_chart.save("chriscox_analysis_20240709.png", dpi=300)
combo_chart

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting
